In [1]:
import os
import sys
from importlib import reload
from pandas import DataFrame, read_csv

C:\Users\user\AppData\Local\Temp\ipykernel_16284\1231245394.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  from pandas import DataFrame, read_csv


Загрузим данные

In [2]:
data = read_csv('../data/refined/wide/blood_chemistry_17.csv', index_col=(0, 1))

In [3]:
import torch
device = 'gpu' if torch.cuda.is_available() else 'cpu'

Сгенерируем граф 6NN

In [4]:
from initializer import KNNInit
from sklearn.preprocessing import RobustScaler
from missing_sampler import UniformMissing
from pipeline import Preprocessor
from misc import columns_str2int

data_columns = data.columns.to_list()
target_columns = ['Potassium',
       'Sodium', 'Creatinine', 'Chloride', 'Urea Nitrogen', 'Bicarbonate',
       'Anion Gap', 'Glucose', 'Magnesium', 'Calcium, Total', 'Phosphate',]
droprate = 0.3



In [184]:
pp = Preprocessor(
    UniformMissing(droprate, None, target_columns, data_cols=data_columns),
    KNNInit(n_neighbors=6),
    scale=True
)
attrs, G, mask = pp.run(data.values, 'graph', fill_value=0)

c:\Users\user\labevent\src\initializer.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(self.data, device=self.device),


Попробуем воспользоваться GCNMF

In [169]:
tmp = torch.tensor(data.to_numpy())
from imputation import GCNmfConv
GCNmfConv(21, 4, n_components=3, dropout=0)(attrs, G.edge_index, mask).size()

torch.Size([2085, 4])

In [8]:
y = attrs[:, -1]
X = attrs[:, columns_str2int(data_columns, target_columns)]

In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import to_torch_csr_tensor
from torch_geometric.nn import SimpleConv

In [275]:
import importlib
import pipeline
import fp
import loss
import missing_sampler
import imputation
import initializer
to_reload = [pipeline, fp, loss, missing_sampler, imputation, initializer]
for i in to_reload:
    importlib.reload(i)

from pipeline import Pipeline
from imputation import GCNmfImputer, GAEMF
from fp import FeaturePropagation
from loss import SeparableMSE
from missing_sampler import UniformMissing

In [276]:
Pipeline(
    Preprocessor(
        UniformMissing(0.9, data.values.shape, target_cols=target_columns, data_cols=data_columns),
        KNNInit(n_neighbors=1),
        scale=True,
    ),
    imputation.GCNmfImputer(
        GAEMF(
            (21, 64, 10),
            (10, 21),
            dropout=.1,
            n_comp=3
        ), 1
    ),
SeparableMSE(True)
).eval(data, 10)

KeyboardInterrupt: 

In [304]:
param_sampler = {'UniformMissing': {
    'droprate': [
        # 0.1, 0.3, 0.5, 0.75,
          .9],
    'data_cols': [data_columns],
    'target_cols': [target_columns],
    }
}
param_init = {
    'KNNInit': {
        'n_neighbors': [2, 4, 5, 8]
    },
    'RandomInit': {
        'random_graph': ['erdos_renyi'],
        'edge_prob': [0.01, 0.05, 0.1, 0.25, 0.5]
    },
    'EpsilonRadiusInit': {
        'radius': [0.1, .5]
    },
    'FeatureCliqueInit': {
        'division_factor': [5, 10],
        'column_indices': [[1]]
    }

}
param_imp = {
    'FeaturePropagation': {
        'n_iterations': [1, 2, 4, 8]
    },
    'GCNmfImputer': {
        'train_epochs': [1, 2,],
        'model': [
            GAEMF(
                (21, 32, 8),
                (8, 21), 
                n_comp=3
            ),
            GAEMF(
                (21, 32, 8),
                (8, 21), 
                dropout=.1,
                n_comp=3
            ),
            GAEMF(
                (21, 32, 8),
                (8, 21), 
                dropout=.1,
                n_comp=3
            ),
        ]
    }
}

from itertools import product
def product_param(d):
    keys, values = zip(*d.items())
    return [dict(zip(keys, v)) for v in product(*values)]


def experimental_settings(sampler: str, initer: str, imputer: str):
    for samp, init, imp in product(
        product_param(param_sampler[sampler]),
        product_param(param_init[initer]),
        product_param(param_imp[imputer]),
    ):
        yield ({'sampler': samp,
               'initer': init,
               'imputer': imp}, 
               {'sampler': sampler,
               'initer': initer,
               'imputer': imputer}, 
               ), Pipeline(
            Preprocessor(
                getattr(missing_sampler, sampler)(**samp),
                getattr(initializer, initer)(**init),
                scale=True
            ),
            getattr(imputation, imputer)(**imp),
            SeparableMSE(True),
        )

In [305]:
results = DataFrame(columns=['type', 'sampler', 'initer',  'imputer', *target_columns])

In [306]:
path = '../data/callbacks'
if not os.path.exists(path):
    os.mkdir('../data/callbacks')

In [307]:
colmask = columns_str2int(data, target_columns)
n = 5
for sampler, initer, imputer in product(
    ['UniformMissing'],
    ['KNNInit', 'EpsilonRadiusInit', 'RandomInit', 'FeatureCliqueInit'],
    [
        'FeaturePropagation', 
     'GCNmfImputer']
):  
    for param, pipeline in experimental_settings(sampler, initer, imputer):
        res = torch.stack(pipeline.eval(data, n))
        header = [
            (param[0][stage], param[1][stage]) for stage in ['sampler', 'initer', 'imputer']
        ]
        results.loc[results.shape[0] + 1] = ['mean'] + header + res.mean(0)[colmask].tolist()
        results.loc[results.shape[0] + 1] = ['std'] + header + res.std(0)[colmask].tolist()
    results.to_csv(os.path.join(path, 'results.csv'))
        
    

Trial:   0%|          | 0/5 [00:00<?, ?it/s]

RuntimeError: bad allocation